# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     |Sofia Anaya Palafox   |
| **Fecha**      |16 de feb del 2026   |
| **Expediente** |738594   |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [1]:
import pandas as pd

file_path = '/content/Motor Trend Car Road Tests.xlsx'
df_mtcars = pd.read_excel(file_path)

display(df_mtcars.head())

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut, cross_val_score
import numpy as np

#preparar os datos X_mt: Todas las columnas EXCEPTO 'model' y 'mpg'
#y_mt: Solo la columna 'mpg'

X_mt = df_mtcars.drop(columns=['model', 'mpg'])
y_mt = df_mtcars['mpg']

#LeaveOneOut(): Crea un método de validación donde se usan 31 para entrenar y se prueba uno

loo = LeaveOneOut()
model = LinearRegression()

# cross val hace la validación cruzada automáticamente
# neg_mean_squared_error usa el error cuadrático medio negativo
scores = cross_val_score(model, X_mt, y_mt, cv=loo, scoring='neg_mean_squared_error')

# lo hace positivo
mse_scores = -scores
# promediamos errores
mean_mse = mse_scores.mean()
# que tanto varian los modelos
std_mse = mse_scores.std()

print(f'MSE promedio (LOOCV): {mean_mse:.4f}')
print(f'Desviación estándar del MSE: {std_mse:.4f}')

MSE promedio (LOOCV): 12.1816
Desviación estándar del MSE: 17.0674


In [9]:
#numpy: Para operaciones con arreglos numéricos
#LinearRegression: El modelo que vamos a usar
#copy(): Creamos una copia para no modificar el original

import numpy as np
from sklearn.linear_model import LinearRegression
df = df_mtcars.copy()

#Si existe la columna 'model' (nombre del auto), la eliminamos
#Los nombres no sirven para predecir numéricamente
if 'model' in df.columns:
    df = df.drop(columns=['model'])

#X: Todas las columnas EXCEPTO mpg (cilindros, peso, etc.)
#y: Solo la columna mpg (consumo)
#n: Cantidad de autos en el dataset
X = df.drop(columns=['mpg']).values
y = df['mpg'].values
n = len(y)

# Lista vacía para guardar los errores
mse_list = []

# Para CADA auto en el dataset (i = 0, 1, 2, ..., 31)
for i in range(n):
    X_train = np.delete(X, i, axis=0) # TODOS los autos EXCEPTO el i-ésimo
    y_train = np.delete(y, i) # Sus consumos correspondiente
    X_test = X[i].reshape(1, -1) # SOLO el auto i-ésimo (en formato fila)
    y_test = y[i]  #Su consumo real

    model = LinearRegression()
    model.fit(X_train, y_train) #el modelo aprende con 31 autos

    y_pred = model.predict(X_test) # Predice el consumo del auto que guardamos
    mse = (y_test - y_pred[0]) ** 2 # Error cuadrático para ESTE auto
    mse_list.append(mse) # Guardamos el error

mse_mean = np.mean(mse_list) # Promedio de los 32 errores
mse_std = np.std(mse_list) # Variabilidad de los errores
print(f"MSE promedio: {mse_mean:.4f}")
print(f"Desviación estándar del MSE: {mse_std:.4f}")

MSE promedio: 12.1816
Desviación estándar del MSE: 17.0674


Interpreta.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3